### Bloc 5.2 – Correction : M1 et phases de cycle (clarification méthodologique)

Dans la cellule précédente, nous avons construit un tableau
`M1_S0_d_by_W_k_cycle_phase.csv` en combinant :

- [M1_S0_all_windows_per_k.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/d_estimates/M1_S0_all_windows_per_k.csv:0:0-0:0) (où `d_hat_k` est **agrégé globalement** par `(W, G, k)` sur toutes les fenêtres),
- `windows_with_cycle_phase.csv` (qui donne la phase de cycle de chaque fenêtre).

Ce qu’il est important de reconnaître explicitement :

- [M1_S0_all_windows_per_k.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/d_estimates/M1_S0_all_windows_per_k.csv:0:0-0:0) **ne contient aucune information par fenêtre individuelle**.
- Le tableau `M1_S0_d_by_W_k_cycle_phase` attribue donc **la même valeur `d_hat_k`** à toutes les phases (`min`, `max`, `rising`, `declining`, …) pour un triplet donné `(W, G, k)` ; seule la fraction de fenêtres dans chaque phase change.
- En conséquence, **il est incorrect** d’utiliser ce tableau pour interpréter une dépendance de `d_M1` aux phases de cycle.  
  Ce serait un *place holder* déguisé : on ré-étiquette un `d` global sans l’avoir recalculé par phase.

**Décision :**

- Nous considérons `M1_S0_d_by_W_k_cycle_phase.csv` comme un artefact purement
  technique (comptage de fenêtres par phase), **non utilisable** pour une analyse
  scientifique du type “M1 pour minima / M1 pour maxima”.
- Nous **n’utiliserons pas** ce tableau pour tirer des conclusions sur `d_M1`
  en fonction de la phase de cycle. Toute interprétation dans ce sens serait
  fictive et contraire à l’esprit du projet (pas de placeholders).

**Plan pour la suite :**

1. **Si nous voulons vraiment analyser `d_M1` par phase de cycle**, il faudra
   définir un bloc dédié (par ex. Bloc 5.3) qui :
   - recalcule M1 **par fenêtre** (ou au moins par sous‑ensemble de fenêtres
     sélectionnées par phase) ;
   - produit un vrai `d_M1(window, phase)` ou `d_M1(W, G, phase)` basé sur des
     calculs effectifs, pas sur un simple ré-étiquetage.
2. Tant que ce recalcul n’est pas fait, nous nous appuierons **uniquement** sur
   des informations réellement disponibles :
   - distributions de fenêtres par phase (fenêtres + `cycle_phase`) ;
   - méthodes qui ont une définition claire à l’échelle considérée
     (M2, M3, M4) pour les analyses par W ou par phase.

À partir de maintenant, toute analyse de `d` ou de \(T_{\log}\) par phase de cycle
sera explicitement limitée à ce qui a été **effectivement recalculé** à ce niveau
(granularité fenêtre / phase) et ne reposera plus sur des agrégats globaux
ré-étiquetés.

### Bloc 5.3 – M1 Levina–Bickel par fenêtre sur TOUTES les fenêtres (tous W, G, phases)

**Objectif.**  
Recalculer la dimension M1 (Levina–Bickel) **pour chaque fenêtre** de la série S0,
en utilisant les vraies données fenêtres (W, G) et les phases de cycle (`cycle_phase`)
issues de `windows_with_cycle_phase.csv`, sans aucun recours à des estimations
globales agrégées.

**Données d’entrée.**

- Série nettoyée S0 :
  - [data_phase2/sunspots_clean/Sunspots_clean.csv](cci:7://file:///c:/Users/zackd/OneDrive/Desktop/Phase2_Tlog_v0.5/SunspotPhase2Tlog/data_phase2/sunspots_clean/Sunspots_clean.csv:0:0-0:0)
- Fenêtres avec métadonnées et phase de cycle :
  - `data_phase2/windows/windows_with_cycle_phase.csv`
- Paramètres M1 (cohérents avec le Bloc 3.3) :
  - `K_MIN = 5`, `K_MAX = 20`.

**Méthode.**

1. Charger `windows_with_cycle_phase.csv` et regrouper les fenêtres par
   `(window_size_months = W, stride_months = G)`.
2. Pour chaque combinaison `(W, G)` :
   - extraire, pour toutes les fenêtres de ce groupe, les vecteurs de taille `W`
     à partir de S0 (`Monthly Mean Total Sunspot Number`), ce qui donne une
     matrice \( X_{W,G} \in \mathbb{R}^{n_{W,G} \times W} \) ;
   - ajuster `K_MAX` si nécessaire pour garantir `K_MAX < n_{W,G}` ;
   - calculer les distances k‑NN (euclidiennes) entre toutes les fenêtres
     du groupe (hors distance à soi-même) ;
   - pour chaque \( k \in \{K_{\min}, \dots, K_{\max}^{\text{eff}}\} \),
     calculer \(\hat{d}_i(k)\) pour chaque fenêtre \(i\) avec la formule
     Levina–Bickel au niveau **par point** :

     \[
     \hat d_i(k)
     =
     \left(
       \frac{1}{k-1}
       \sum_{j=1}^{k-1}
         \log\frac{r_{i,k}}{r_{i,j}}
     \right)^{-1}
     \]

     où \(r_{i,1} \le \dots \le r_{i,k}\) sont les distances triées aux
     k plus proches voisins (hors i).

3. Assembler un tableau global avec, pour chaque fenêtre et chaque k :

   - `series`, `W`, `G`, `window_id`, `cycle_phase`,
   - `k`, `d_hat_i_k`.

4. Construire un résumé agrégé par `(W, G, cycle_phase, k)` :

   - `d_mean`, `d_std`, `n_windows`.

**Sorties.**

- Tableau détaillé par fenêtre & k :

  - `data_phase2/d_estimates_by_phase/M1_S0_per_window_all_WG.csv`

- Résumé par `(W, G, cycle_phase, k)` :

  - `data_phase2/d_estimates_by_phase/M1_S0_summary_by_W_G_phase_k.csv`

Ces fichiers fournissent des estimations M1 réellement calculées **par fenêtre**
pour tous les W et G du dataset, avec l’étiquette de phase de cycle. Toute
analyse de la dépendance de `d_M1` aux phases (min, max, rising, declining, …)
devra se baser sur ces fichiers, sans utiliser les anciens agrégats globaux.

In [36]:
# Bloc 5.3 – M1 Levina–Bickel par fenêtre pour toutes les fenêtres (tous W, G, phases)

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.neighbors import NearestNeighbors

# 5.3.1 – Paramètres M1 (cohérents avec le Bloc 3.3)
K_MIN = 5
K_MAX = 20

if K_MIN < 2:
    raise ValueError("K_MIN doit être >= 2 pour la formule Levina–Bickel.")
if K_MIN >= K_MAX:
    raise ValueError("K_MIN doit être strictement inférieur à K_MAX.")

# 5.3.2 – Chargement de la série S0 (Sunspots_clean)
DATA_PHASE2_CLEAN_DIR = PHASE2_ROOT / "data_phase2" / "sunspots_clean"
SUNSPOTS_CLEAN_CSV_PATH = DATA_PHASE2_CLEAN_DIR / "Sunspots_clean.csv"

if not SUNSPOTS_CLEAN_CSV_PATH.exists():
    raise FileNotFoundError(
        f"Sunspots_clean.csv est introuvable à : {SUNSPOTS_CLEAN_CSV_PATH}.\n"
        "Assurez-vous d'avoir exécuté les blocs 1.3 et 1.4."
    )

df_s0 = pd.read_csv(SUNSPOTS_CLEAN_CSV_PATH, parse_dates=["Date"])
df_s0 = df_s0.sort_values("Date").reset_index(drop=True)

value_col = "Monthly Mean Total Sunspot Number"
df_ts = df_s0.set_index("Date")[[value_col]]

# 5.3.3 – Chargement des fenêtres avec phase de cycle
WINDOWS_DIR = PHASE2_ROOT / "data_phase2" / "windows"
WINDOWS_PHASE_PATH = WINDOWS_DIR / "windows_with_cycle_phase.csv"

if not WINDOWS_PHASE_PATH.exists():
    raise FileNotFoundError(
        f"windows_with_cycle_phase.csv est introuvable à : {WINDOWS_PHASE_PATH}.\n"
        "Assurez-vous d'avoir exécuté le Bloc 5.1."
    )

df_windows_phase = pd.read_csv(
    WINDOWS_PHASE_PATH,
    parse_dates=["start_date", "end_date", "window_center_date"],
)

required_win_cols = [
    "window_id",
    "window_size_months",
    "stride_months",
    "start_index",
    "end_index",
    "n_points",
    "cycle_phase",
]
missing_win = [c for c in required_win_cols if c not in df_windows_phase.columns]
if missing_win:
    raise RuntimeError(
        f"Colonnes manquantes dans windows_with_cycle_phase.csv : {missing_win}"
    )

# 5.3.4 – Fonction Levina–Bickel par point pour un k donné
def levina_bickel_d_hat_per_point(distances_sorted, k):
    """
    distances_sorted : array (n_samples, K_max_eff)
        Colonnes = distances r_{i,1} <= ... <= r_{i,K_max_eff}
        (hors distance à soi-même).
    k : entier (2 <= k <= K_max_eff)
    Retourne un vecteur d_hat_i(k) de longueur n_samples.
    """
    n_samples, K_max_eff_local = distances_sorted.shape
    if k < 2 or k > K_max_eff_local:
        raise ValueError(f"k={k} doit être dans [2, {K_max_eff_local}]")

    r_k = distances_sorted[:, k - 1]  # r_{i,k}
    r_1_to_kminus1 = distances_sorted[:, : k - 1]  # r_{i,1..k-1}

    eps = 1e-12
    r_k_safe = np.maximum(r_k, eps)
    r_1_to_kminus1_safe = np.maximum(r_1_to_kminus1, eps)

    logs = np.log(r_k_safe[:, None] / r_1_to_kminus1_safe)  # shape (n_samples, k-1)
    sums = logs.sum(axis=1) / (k - 1)

    sums_safe = np.maximum(sums, eps)
    d_hat_i_k = 1.0 / sums_safe

    return d_hat_i_k

# 5.3.5 – Boucle sur toutes les combinaisons (W, G) pour calculer M1 par fenêtre
records = []

group_cols = ["window_size_months", "stride_months"]
groups = df_windows_phase.groupby(group_cols, sort=True)

print("Combinaisons (W, G) trouvées :", groups.size().index.tolist())

for (W, G), df_group in groups:
    df_group = df_group.sort_values("window_center_date").reset_index(drop=True)
    n_windows_group = df_group.shape[0]

    print(f"\n=== Traitement M1 pour (W={W}, G={G}) ===")
    print(f"Nombre de fenêtres dans ce groupe : {n_windows_group}")

    if n_windows_group <= K_MIN:
        print(
            f"  -> n_windows={n_windows_group} <= K_MIN={K_MIN}, "
            "groupe ignoré pour M1 (pas assez de voisins)."
        )
        continue

    # Construction de la matrice X (fenêtres × W)
    X_list = []
    window_ids = []
    phases = []

    for _, row in df_group.iterrows():
        start_idx = int(row["start_index"])
        end_idx = int(row["end_index"])
        n_points = int(row["n_points"])

        if n_points != W:
            raise RuntimeError(
                f"Incohérence n_points={n_points} pour window_id={row['window_id']} "
                f"avec W={W}."
            )

        window_values = df_ts.iloc[start_idx : end_idx + 1][value_col].to_numpy()

        if window_values.shape[0] != W:
            raise RuntimeError(
                f"Taille de fenêtre inattendue pour window_id={row['window_id']}: "
                f"{window_values.shape[0]} au lieu de {W}."
            )

        X_list.append(window_values)
        window_ids.append(int(row["window_id"]))
        phases.append(str(row["cycle_phase"]))

    X = np.vstack(X_list)
    window_ids = np.array(window_ids)
    phases = np.array(phases)

    print(f"  Matrice X shape = {X.shape} (n_fenêtres, W).")

    # Ajustement de K_MAX par groupe
    K_MAX_eff = min(K_MAX, n_windows_group - 1)
    if K_MAX_eff < K_MAX:
        print(
            f"  K_MAX réduit de {K_MAX} à {K_MAX_eff} car n_fenêtres={n_windows_group} "
            "est insuffisant pour K_MAX initial."
        )

    if K_MAX_eff < K_MIN:
        print(
            f"  -> K_MAX_eff={K_MAX_eff} < K_MIN={K_MIN}, groupe ignoré pour M1."
        )
        continue

    # 5.3.6 – Calcul des distances k-NN pour ce groupe
    n_neighbors = K_MAX_eff + 1  # +1 pour inclure la distance à soi-même

    nn = NearestNeighbors(
        n_neighbors=n_neighbors,
        metric="euclidean",
    )
    nn.fit(X)
    distances_full, indices_full = nn.kneighbors(X)
    distances = distances_full[:, 1:]  # on enlève la distance à soi-même

    assert distances.shape[1] == K_MAX_eff

    # 5.3.7 – Calcul d_hat_i(k) pour tous k
    for k in range(K_MIN, K_MAX_eff + 1):
        d_hat_i_k = levina_bickel_d_hat_per_point(distances, k)

        for idx in range(n_windows_group):
            records.append(
                {
                    "series": "S0",
                    "W": int(W),
                    "G": int(G),
                    "window_id": int(window_ids[idx]),
                    "cycle_phase": phases[idx],
                    "k": int(k),
                    "d_hat_i_k": float(d_hat_i_k[idx]),
                }
            )

# 5.3.8 – Assemblage du DataFrame global
if not records:
    raise RuntimeError(
        "Aucune estimation M1 par fenêtre n'a été produite (records est vide)."
    )

df_m1_per_window = pd.DataFrame(records)

print("\nAperçu global de M1 par fenêtre (tous W, G, phases) :")
display(df_m1_per_window.head())

# 5.3.9 – Résumé par (W, G, cycle_phase, k)
df_m1_summary = (
    df_m1_per_window
    .groupby(["W", "G", "cycle_phase", "k"])
    .agg(
        d_mean=("d_hat_i_k", "mean"),
        d_std=("d_hat_i_k", "std"),
        n_windows=("window_id", "nunique"),
    )
    .reset_index()
    .sort_values(["W", "G", "cycle_phase", "k"])
)

print("\nRésumé M1 par (W, G, cycle_phase, k) :")
display(df_m1_summary.head(20))

# 5.3.10 – Sauvegarde des résultats
D_EST_PHASE_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_by_phase"
D_EST_PHASE_DIR.mkdir(parents=True, exist_ok=True)

M1_PER_WINDOW_PATH = D_EST_PHASE_DIR / "M1_S0_per_window_all_WG.csv"
M1_SUMMARY_PHASE_K_PATH = D_EST_PHASE_DIR / "M1_S0_summary_by_W_G_phase_k.csv"

df_m1_per_window.to_csv(M1_PER_WINDOW_PATH, index=False)
df_m1_summary.to_csv(M1_SUMMARY_PHASE_K_PATH, index=False)

print("\nFichiers M1 par fenêtre et résumés par (W,G,phase,k) écrits à :")
print(f"  - {M1_PER_WINDOW_PATH}")
print(f"  - {M1_SUMMARY_PHASE_K_PATH}")

# 5.3.11 – Logging pour audit
phase_counts_global = df_m1_per_window["cycle_phase"].value_counts().to_dict()

log_message(
    "INFO",
    (
        "M1 Levina–Bickel recalculé par fenêtre pour toutes les combinaisons (W,G) "
        "avec étiquette de phase de cycle. "
        f"Résultats sauvegardés dans {M1_PER_WINDOW_PATH.name} et "
        f"{M1_SUMMARY_PHASE_K_PATH.name}."
    ),
    block="BLOC_5.3",
)
log_metric(
    "M1_per_window_all_WG_rows",
    int(df_m1_per_window.shape[0]),
    extra={
        "n_rows_summary": int(df_m1_summary.shape[0]),
        "K_min": int(K_MIN),
        "K_max": int(K_MAX),
        "phase_counts_global": phase_counts_global,
    },
)

Combinaisons (W, G) trouvées : [(60, 1), (60, 6), (60, 12), (132, 1), (132, 6), (132, 12), (264, 1), (264, 6), (264, 12)]

=== Traitement M1 pour (W=60, G=1) ===
Nombre de fenêtres dans ce groupe : 3206
  Matrice X shape = (3206, 60) (n_fenêtres, W).

=== Traitement M1 pour (W=60, G=6) ===
Nombre de fenêtres dans ce groupe : 535
  Matrice X shape = (535, 60) (n_fenêtres, W).

=== Traitement M1 pour (W=60, G=12) ===
Nombre de fenêtres dans ce groupe : 268
  Matrice X shape = (268, 60) (n_fenêtres, W).

=== Traitement M1 pour (W=132, G=1) ===
Nombre de fenêtres dans ce groupe : 3134
  Matrice X shape = (3134, 132) (n_fenêtres, W).

=== Traitement M1 pour (W=132, G=6) ===
Nombre de fenêtres dans ce groupe : 523
  Matrice X shape = (523, 132) (n_fenêtres, W).

=== Traitement M1 pour (W=132, G=12) ===
Nombre de fenêtres dans ce groupe : 262
  Matrice X shape = (262, 132) (n_fenêtres, W).

=== Traitement M1 pour (W=264, G=1) ===
Nombre de fenêtres dans ce groupe : 3002
  Matrice X shape = (3

,series,W,G,window_id,cycle_phase,k,d_hat_i_k
0,S0,60,1,0,declining,5,9.841803
1,S0,60,1,1,declining,5,10.231334
2,S0,60,1,2,declining,5,12.984415
3,S0,60,1,3,declining,5,15.419666
4,S0,60,1,4,declining,5,26.045554



Résumé M1 par (W, G, cycle_phase, k) :


,W,G,cycle_phase,k,d_mean,d_std,n_windows
0,60,1,declining,5,26.404783,20.929353,1542
1,60,1,declining,6,23.510060,16.028486,1542
2,60,1,declining,7,21.942043,13.438563,1542
3,60,1,declining,8,20.878843,11.935334,1542
4,60,1,declining,9,20.064124,10.662644,1542
5,60,1,declining,10,19.396500,9.897699,1542
6,60,1,declining,11,18.788644,9.147751,1542
7,60,1,declining,12,18.355276,8.511852,1542
8,60,1,declining,13,17.932092,7.991814,1542
9,60,1,declining,14,17.679886,7.609453,1542



Fichiers M1 par fenêtre et résumés par (W,G,phase,k) écrits à :
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_S0_per_window_all_WG.csv
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_S0_summary_by_W_G_phase_k.csv
[STEP=34][INFO][BLOC_5.3] M1 Levina–Bickel recalculé par fenêtre pour toutes les combinaisons (W,G) avec étiquette de phase de cycle. Résultats sauvegardés dans M1_S0_per_window_all_WG.csv et M1_S0_summary_by_W_G_phase_k.csv.
[METRIC][M1_per_window_all_WG_rows] = 186912 (step=34)


### Bloc 5.3 – Audit du recalcul M1 par fenêtre et cadre d’analyse minima/maxima

**But de l’audit.**
- Confirmer que M1 (Levina–Bickel) a été recalculé **par fenêtre**, pour **toutes** les paires \((W,G)\) et phases (`min`, `max`, `rising`, `declining`, …), sans réutiliser d’agrégat global.
- Établir les règles d’analyse pour la comparaison `min` vs `max` sans « résumé arbitraire » de M1 à ce stade.

**Provenance des données (rappel court).**
- Série réelle S0: `data_phase2/sunspots_clean/Sunspots_clean.csv` (dérivée de `sunspots.zip` via Blocs 1.0–1.4).
- Fenêtres: `data_phase2/windows/window_definitions.csv` (glissement déterministe sur S0).
- Phases de cycle: `data_phase2/windows/windows_with_cycle_phase.csv` (détection heuristique min/max, puis annotation des fenêtres).
- Ancien agrégat global M1: `data_phase2/d_estimates/M1_S0_all_windows_per_k.csv` (moyennes par (W,G,k), non utilisé pour l’analyse de phase).

**Ce que fait exactement le Bloc 5.3 (exécuté).**
- Pour chaque \((W,G)\) trouvé dans `windows_with_cycle_phase.csv` (ex. `(60,1)`, `(60,6)`, `(60,12)`, `(132,1)`, `(132,6)`, `(132,12)`, `(264,1)`, `(264,6)`, `(264,12)`):
  - Reconstitution explicite des vecteurs de fenêtres depuis S0 par indices `[start_index:end_index]` et vérification `n_points == W`.
  - Calcul des distances k-NN (euclidiennes) entre fenêtres du groupe, puis application Levina–Bickel par **fenêtre** pour chaque `k ∈ {5,…,20}`.
  - Assemblage global: colonnes `series, W, G, window_id, cycle_phase, k, d_hat_i_k`.
- Sorties produites:
  - `data_phase2/d_estimates_by_phase/M1_S0_per_window_all_WG.csv` (par fenêtre & k).
  - `data_phase2/d_estimates_by_phase/M1_S0_summary_by_W_G_phase_k.csv` (moyennes/écarts-types par (W,G,phase,k)).
- Contrôle d’exhaustivité: le compteur de lignes correspond au nombre total de fenêtres × nombre de k (aucune fenêtre omise). Aucun placeholder.

**Position méthodologique (important).**
- Aucune « règle de résumé » de M1 n’est imposée maintenant. On n’agrège pas M1 en une seule valeur tant que les diagnostics `min` vs `max` n’ont pas été faits, par W et par k.
- Hypothèse de travail (à tester, pas à imposer):
  - M1 serait plus robuste/pertinent aux minima.
  - M2 (PR/PCA) serait plus robuste/pertinent aux maxima.
- Objectif Phase 2: vérifier empiriquement cette hypothèse sur S0, par W, avec incertitudes, **avant** toute calibration ou bascule vers T_log et Phase 3.

---

#### Plan opérationnel pour minima/maxima (sans conclusion prématurée)

1) Diagnostics M1 `min` vs `max` (Bloc 5.4)
- Pour chaque `(W,G)` et pour chaque `k ∈ {5,…,20}`:
  - Utiliser des sous-ensembles **non chevauchants** de fenêtres par phase (greedy: imposer un espacement ≥ W entre `start_index`) pour réduire l’autocorrélation.
  - Comparer distributions `d_hat_i_k` entre `min` et `max` par tests non paramétriques et/ou permutation tests (différence de médianes ou de moyennes), avec taille d’effet (Cliff’s delta, ou d de Cohen si pertinent).
  - Contrôle du risque multiplicatif (plusieurs k, plusieurs W): FDR (Benjamini–Hochberg) ou Holm.
  - Exiger des effectifs minimums après non-chevauchement; sinon marquer « non concluant ».
  - Produire des figures `k → d_mean ± d_std` par phase, et tables complètes CSV (pas de résumé unique imposé).

2) Recalcul M2 par phase (Bloc 5.5)
- Pour chaque `(W,G,phase)`: construire la matrice des fenêtres de la phase, centrer/standardiser par axe des features si nécessaire, calculer participation ratio / dimension PR à partir du spectre de covariance.
- Bootstrap sur fenêtres (avec non-chevauchement ou bloc bootstrap) pour `d_PR` et intervalles (80/90%).
- Export CSV `M2_S0_PR_by_W_G_phase.csv` + diagnostics min vs max analogues à M1.

3) M3 spectral par phase (Bloc 5.6)
- Joindre `M3_S0_spectral_per_window.csv` aux phases; si couverture partielle, étendre le calcul si besoin.
- Résumer `beta` (ou `d_s(beta)`) par `(W,G,phase)` avec incertitudes; diagnostics min vs max.

4) M4 (ancre externe) et T_log (Bloc 5.7–5.8)
- M4 reste une **référence globale** (Phase 1) – pas de dérivé par phase.
- Calculer `T_log(n_W, d)` par phase **uniquement après** validation des diagnostics M1/M2/M3.
- Décrire les régimes (divergence, criticité, saturation) par phase et par W avec incertitudes.

5) Préparation Phase 3 (Bloc 5.9)
- Si les diagnostics confirment « M1 pour minima » et « M2 pour maxima »:
  - Définir modules `min` et `max` (M1/M2) + module global; quantifier incertitudes (bootstrap/permutation).
  - Décrire un mécanisme de détection (à partir d’un nouveau segment) pour inférer le domaine (`min`, `max`, `rising`, `declining`) et estimer quand un basculement est probable.

Cet audit verrouille la traçabilité du recalcul M1 par fenêtre, et cadre l’analyse `min`/`max` sans imposer de résumé arbitraire ni conclusion anticipée. Toute décision ultérieure (calibration, T_log, Phase 3) sera conditionnée aux résultats empiriques multi‑méthodes (M1–M4).

### Bloc 5.4 – Diagnostics M1 minima vs maxima (sans résumé arbitraire)

**Objectif.** Vérifier empiriquement l’hypothèse de travail "M1 est plus pertinent/robuste aux minima" sans imposer de règle de résumé de M1. On compare directement les distributions `d_hat_i_k` (par fenêtre) entre `min` et `max`, pour chaque `(W,G)` et chaque `k ∈ {5,…,20}`.

**Méthode (principes).**
- Sélection de fenêtres **non chevauchantes** au sein de chaque phase, via un algorithme glouton sur `start_index` avec pas minimal `≥ W`.
- Comparaison `min` vs `max` par `k` avec:
  - différences de moyennes/medians,
  - **taille d’effet** (Cliff’s delta),
  - **test par permutation** (deux côtés) sur la différence de moyennes (n_perm=2000, seed déterministe à partir de `GLOBAL_SEED`).
- **Contrôle FDR** (Benjamini–Hochberg) multi-tests sur l’ensemble des p-valeurs (tous `(W,G,k)`).
- Aucun *résumé* de M1 n’est choisi ici: on laisse les résultats par `k` pour juger la stabilité ou non.

**Sorties.**
- `data_phase2/d_estimates_by_phase/M1_min_max_nonoverlap_samples.csv`:
  - Fenêtres retenues (non chevauchantes) par `(W,G,phase)`: `window_id, start_index, end_index`.
- `data_phase2/d_estimates_by_phase/M1_min_vs_max_nonoverlap_stats.csv`:
  - Par `(W,G,k)`: `n_min, n_max, mean_min, mean_max, median_min, median_max, std_min, std_max, diff_means, cliffs_delta, p_perm, q_fdr, significant_fdr_0_05`.

Ces diagnostics permettront de statuer si M1 discrimine effectivement `min` vs `max` et pour quels `(W,G,k)`, **avant** toute calibration ou rapprochement avec M2/M3/M4.

In [37]:
# Bloc 5.4 – Diagnostics M1 minima vs maxima (code)

import numpy as np
import pandas as pd
from pathlib import Path

# Entrées
D_EST_PHASE_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_by_phase"
M1_PER_WINDOW_PATH = D_EST_PHASE_DIR / "M1_S0_per_window_all_WG.csv"
WINDOWS_PHASE_PATH = PHASE2_ROOT / "data_phase2" / "windows" / "windows_with_cycle_phase.csv"

if not M1_PER_WINDOW_PATH.exists():
    raise FileNotFoundError(f"Introuvable: {M1_PER_WINDOW_PATH}")
if not WINDOWS_PHASE_PATH.exists():
    raise FileNotFoundError(f"Introuvable: {WINDOWS_PHASE_PATH}")

# Chargements
cols_needed_win = [
    "window_id",
    "window_size_months",
    "stride_months",
    "start_index",
    "end_index",
    "n_points",
    "cycle_phase",
]
df_m1 = pd.read_csv(M1_PER_WINDOW_PATH)
df_win = pd.read_csv(WINDOWS_PHASE_PATH, usecols=cols_needed_win)

# Jointure pour récupérer start/end/n_points
left_on = ["window_id", "W", "G", "cycle_phase"]
right_on = ["window_id", "window_size_months", "stride_months", "cycle_phase"]
df = df_m1.merge(df_win, left_on=left_on, right_on=right_on, how="inner")

# Filtre phases min/max
PHASES = ["min", "max"]
df = df[df["cycle_phase"].isin(PHASES)].copy()

# Fonctions utilitaires

def select_nonoverlap(df_phase, W):
    df_phase = df_phase.sort_values("start_index")
    selected_ids = []
    last_end = -1
    for _, r in df_phase.iterrows():
        if int(r["start_index"]) > last_end:
            selected_ids.append(int(r["window_id"]))
            last_end = int(r["start_index"]) + int(W) - 1
    return selected_ids


def cliffs_delta(x, y):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    n = x.size
    m = y.size
    if n == 0 or m == 0:
        return np.nan
    y_sorted = np.sort(y)
    less = 0
    greater = 0
    for xi in x:
        left = np.searchsorted(y_sorted, xi, side="left")
        right = np.searchsorted(y_sorted, xi, side="right")
        less += left
        greater += m - right
    return (less - greater) / (n * m)


def perm_test_diff_means(x, y, n_perm=2000, rng=None):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    n_x = x.size
    n_y = y.size
    if n_x == 0 or n_y == 0:
        return np.nan
    combined = np.concatenate([x, y])
    obs = x.mean() - y.mean()
    if rng is None:
        rng = np.random.default_rng(42)
    cnt = 0
    for _ in range(n_perm):
        perm = rng.permutation(combined)
        x_perm = perm[:n_x]
        y_perm = perm[n_x:]
        diff = x_perm.mean() - y_perm.mean()
        if abs(diff) >= abs(obs):
            cnt += 1
    p = (cnt + 1) / (n_perm + 1)
    return p


def bh_fdr(pvals):
    pvals = np.asarray(pvals, dtype=float)
    m = len(pvals)
    order = np.argsort(pvals)
    ranked = pvals[order]
    q = ranked * m / (np.arange(m) + 1)
    q = np.minimum.accumulate(q[::-1])[::-1]
    q_full = np.empty_like(q)
    q_full[order] = np.minimum(q, 1.0)
    return q_full

# Paramètres
MIN_PER_PHASE = 10
rng = np.random.default_rng(int(GLOBAL_SEED) + 5403 if "GLOBAL_SEED" in globals() else 5403)

# Sélection non chevauchante et calculs
records_sel = []
records_stats = []

for (W, G), dfg in df.groupby(["W", "G"], sort=True):
    W = int(W)
    G = int(G)

    dfg_min = dfg[dfg["cycle_phase"] == "min"].copy()
    dfg_max = dfg[dfg["cycle_phase"] == "max"].copy()

    sel_min_ids = select_nonoverlap(dfg_min[["window_id", "start_index"]].drop_duplicates("window_id"), W)
    sel_max_ids = select_nonoverlap(dfg_max[["window_id", "start_index"]].drop_duplicates("window_id"), W)

    for order, wid in enumerate(sel_min_ids):
        r = dfg_min[dfg_min["window_id"] == wid].iloc[0]
        records_sel.append({
            "W": W, "G": G, "phase": "min",
            "window_id": int(wid),
            "start_index": int(r["start_index"]),
            "end_index": int(r["end_index"]),
            "n_points": int(r["n_points"]),
            "selected_order": int(order)
        })
    for order, wid in enumerate(sel_max_ids):
        r = dfg_max[dfg_max["window_id"] == wid].iloc[0]
        records_sel.append({
            "W": W, "G": G, "phase": "max",
            "window_id": int(wid),
            "start_index": int(r["start_index"]),
            "end_index": int(r["end_index"]),
            "n_points": int(r["n_points"]),
            "selected_order": int(order)
        })

    k_values = sorted(dfg["k"].unique())
    for k in k_values:
        dm = dfg_min[(dfg_min["window_id"].isin(sel_min_ids)) & (dfg_min["k"] == k)]["d_hat_i_k"].to_numpy()
        dx = dfg_max[(dfg_max["window_id"].isin(sel_max_ids)) & (dfg_max["k"] == k)]["d_hat_i_k"].to_numpy()

        n_min = int(dm.size)
        n_max = int(dx.size)
        if n_min >= MIN_PER_PHASE and n_max >= MIN_PER_PHASE:
            mean_min = float(np.mean(dm))
            mean_max = float(np.mean(dx))
            median_min = float(np.median(dm))
            median_max = float(np.median(dx))
            std_min = float(np.std(dm, ddof=1)) if n_min > 1 else 0.0
            std_max = float(np.std(dx, ddof=1)) if n_max > 1 else 0.0
            diff_means = mean_min - mean_max
            cd = float(cliffs_delta(dm, dx))
            p_perm = float(perm_test_diff_means(dm, dx, n_perm=2000, rng=rng))
        else:
            mean_min = median_min = std_min = np.nan
            mean_max = median_max = std_max = np.nan
            diff_means = np.nan
            cd = np.nan
            p_perm = np.nan

        records_stats.append({
            "W": W, "G": G, "k": int(k),
            "n_min": n_min, "n_max": n_max,
            "mean_min": mean_min, "mean_max": mean_max,
            "median_min": median_min, "median_max": median_max,
            "std_min": std_min, "std_max": std_max,
            "diff_means": diff_means,
            "cliffs_delta": cd,
            "p_perm": p_perm,
        })

# DataFrames de sortie
sel_df = pd.DataFrame(records_sel)
stats_df = pd.DataFrame(records_stats)

# FDR (BH) sur les p-valeurs définies
mask_valid = stats_df["p_perm"].notna().values
q_vals = np.full(stats_df.shape[0], np.nan)
if mask_valid.sum() > 0:
    q_vals[mask_valid] = bh_fdr(stats_df.loc[mask_valid, "p_perm"].to_numpy())
stats_df["q_fdr"] = q_vals
stats_df["significant_fdr_0_05"] = stats_df["q_fdr"].apply(lambda v: bool(v <= 0.05) if pd.notna(v) else False)

# Sauvegardes
OUT_SEL = D_EST_PHASE_DIR / "M1_min_max_nonoverlap_samples.csv"
OUT_STATS = D_EST_PHASE_DIR / "M1_min_vs_max_nonoverlap_stats.csv"
sel_df.to_csv(OUT_SEL, index=False)
stats_df.to_csv(OUT_STATS, index=False)

print("Échantillons non chevauchants sauvegardés:")
print(f"  - {OUT_SEL}")
print("Statistiques min vs max (M1) sauvegardées:")
print(f"  - {OUT_STATS}")

log_message(
    "INFO",
    (
        "Diagnostics M1 min vs max (non chevauchants) terminés. "
        f"Écrits: {OUT_SEL.name}, {OUT_STATS.name}."
    ),
    block="BLOC_5.4",
)
log_metric(
    "M1_min_max_nonoverlap_total_selected",
    int(sel_df.shape[0]),
)
log_metric(
    "M1_min_max_nonoverlap_stats_rows",
    int(stats_df.shape[0]),
)

Échantillons non chevauchants sauvegardés:
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_min_max_nonoverlap_samples.csv
Statistiques min vs max (M1) sauvegardées:
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_min_vs_max_nonoverlap_stats.csv
[STEP=35][INFO][BLOC_5.4] Diagnostics M1 min vs max (non chevauchants) terminés. Écrits: M1_min_max_nonoverlap_samples.csv, M1_min_vs_max_nonoverlap_stats.csv.
[METRIC][M1_min_max_nonoverlap_total_selected] = 317 (step=35)
[METRIC][M1_min_max_nonoverlap_stats_rows] = 144 (step=35)


### Bloc 5.4.1 – Synthèse M1 min vs max (non chevauchants)

Objectif: résumer, **sans conclure prématurément**, où M1 discrimine `min` vs `max` après FDR, pour chaque `(W,G)` et pour quels `k`.

Ce bloc:
- filtre les lignes significatives (q_FDR ≤ 0.05) du fichier de stats 5.4;
- exporte les lignes significatives complètes (par `(W,G,k)`);
- produit un résumé par `(W,G)` (nombre de tests, nombre significatifs, fraction, direction majoritaire via `diff_means`, moyenne de `cliffs_delta`, et bornes `k` min/max des ks significatifs).

Aucune agrégation de M1 en une seule valeur n’est effectuée ici. Les résultats servent uniquement de guide pour la suite (M2/M3 par phase, puis synthèse multi‑méthodes).

In [38]:
# Bloc 5.4.1 – Synthèse M1 min vs max (non chev.) – code

import numpy as np
import pandas as pd
from pathlib import Path

D_EST_PHASE_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_by_phase"
STATS_PATH = D_EST_PHASE_DIR / "M1_min_vs_max_nonoverlap_stats.csv"

if not STATS_PATH.exists():
    raise FileNotFoundError(f"Introuvable: {STATS_PATH}")

stats_df = pd.read_csv(STATS_PATH)

# Filtre significatif après FDR
sig_df = stats_df[(stats_df["q_fdr"].notna()) & (stats_df["q_fdr"] <= 0.05)].copy()
SIG_PATH = D_EST_PHASE_DIR / "M1_min_vs_max_nonoverlap_stats_significant.csv"
sig_df.to_csv(SIG_PATH, index=False)

# Résumé par (W,G)
summary_records = []
for (W, G), dfg in stats_df.groupby(["W", "G"], sort=True):
    dfg_sig = sig_df[(sig_df["W"] == W) & (sig_df["G"] == G)]
    total_tests = int(dfg.shape[0])
    sig_tests = int(dfg_sig.shape[0])
    frac_sig = (sig_tests / total_tests) if total_tests > 0 else np.nan

    if sig_tests > 0:
        mean_diff = float(dfg_sig["diff_means"].mean())
        dir_label = "min>max" if mean_diff > 0 else ("min<max" if mean_diff < 0 else "neutral")
        cliffs_mean = float(dfg_sig["cliffs_delta"].mean()) if dfg_sig["cliffs_delta"].notna().any() else np.nan
        cliffs_abs_mean = float(np.abs(dfg_sig["cliffs_delta"]).mean()) if dfg_sig["cliffs_delta"].notna().any() else np.nan
        k_min = int(dfg_sig["k"].min())
        k_max = int(dfg_sig["k"].max())
        k_count = int(dfg_sig["k"].nunique())
    else:
        mean_diff = np.nan
        dir_label = "no_sig"
        cliffs_mean = np.nan
        cliffs_abs_mean = np.nan
        k_min = np.nan
        k_max = np.nan
        k_count = 0

    summary_records.append({
        "W": int(W),
        "G": int(G),
        "total_tests": total_tests,
        "sig_tests": sig_tests,
        "frac_sig": frac_sig,
        "direction_majority": dir_label,
        "diff_means_mean_sig": mean_diff,
        "cliffs_delta_mean_sig": cliffs_mean,
        "cliffs_delta_abs_mean_sig": cliffs_abs_mean,
        "k_min_sig": k_min,
        "k_max_sig": k_max,
        "k_count_sig": k_count,
    })

summary_df = pd.DataFrame(summary_records).sort_values(["W", "G"]).reset_index(drop=True)

SUMMARY_PATH = D_EST_PHASE_DIR / "M1_min_vs_max_nonoverlap_summary_by_WG.csv"
summary_df.to_csv(SUMMARY_PATH, index=False)

print("Fichiers de synthèse écrits:")
print(f"  - Lignes significatives: {SIG_PATH}")
print(f"  - Résumé par (W,G):     {SUMMARY_PATH}")

print("\nAperçu du résumé par (W,G):")
display(summary_df)

log_message(
    "INFO",
    (
        "Synthèse M1 min vs max (non chev.) terminée. "
        f"Écrits: {SIG_PATH.name}, {SUMMARY_PATH.name}."
    ),
    block="BLOC_5.4.1",
)
log_metric("M1_min_max_sig_rows", int(sig_df.shape[0]))
log_metric("M1_min_max_summary_rows", int(summary_df.shape[0]))

Fichiers de synthèse écrits:
  - Lignes significatives: C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_min_vs_max_nonoverlap_stats_significant.csv
  - Résumé par (W,G):     C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_min_vs_max_nonoverlap_summary_by_WG.csv

Aperçu du résumé par (W,G):


,W,G,total_tests,sig_tests,frac_sig,direction_majority,diff_means_mean_sig,cliffs_delta_mean_sig,cliffs_delta_abs_mean_sig,k_min_sig,k_max_sig,k_count_sig
0,60,1,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
1,60,6,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
2,60,12,16,11,0.6875,min<max,-2.418443,-0.577879,0.577879,8.0,20.0,11
3,132,1,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
4,132,6,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
5,132,12,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
6,264,1,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
7,264,6,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0
8,264,12,16,0,0.0000,no_sig,NaN,NaN,NaN,NaN,NaN,0


[STEP=36][INFO][BLOC_5.4.1] Synthèse M1 min vs max (non chev.) terminée. Écrits: M1_min_vs_max_nonoverlap_stats_significant.csv, M1_min_vs_max_nonoverlap_summary_by_WG.csv.
[METRIC][M1_min_max_sig_rows] = 11 (step=36)
[METRIC][M1_min_max_summary_rows] = 9 (step=36)


In [39]:
# Bloc 5.4.2 – Audit des effectifs non chevauchants par (W,G,phase) – code

import pandas as pd
from pathlib import Path

D_EST_PHASE_DIR = PHASE2_ROOT / "data_phase2" / "d_estimates_by_phase"
SEL_PATH = D_EST_PHASE_DIR / "M1_min_max_nonoverlap_samples.csv"

if not SEL_PATH.exists():
    raise FileNotFoundError(f"Introuvable: {SEL_PATH}")

sel_df = pd.read_csv(SEL_PATH)

counts_df = (
    sel_df.groupby(["W", "G", "phase"], as_index=False)
    .agg(n_selected=("window_id", "nunique"))
    .sort_values(["W", "G", "phase"]).reset_index(drop=True)
)

OUT_COUNTS = D_EST_PHASE_DIR / "M1_nonoverlap_counts_by_W_G_phase.csv"
counts_df.to_csv(OUT_COUNTS, index=False)

print("Effectifs non chevauchants par (W,G,phase) écrits:")
print(f"  - {OUT_COUNTS}")

display(counts_df)

log_message(
    "INFO",
    (
        "Audit des effectifs non chevauchants M1 min/max par (W,G,phase) terminé. "
        f"Écrit: {OUT_COUNTS.name}."
    ),
    block="BLOC_5.4.2",
)
log_metric(
    "M1_nonoverlap_counts_rows",
    int(counts_df.shape[0]),
)

Effectifs non chevauchants par (W,G,phase) écrits:
  - C:\Users\zackd\OneDrive\Desktop\Phase2_Tlog_v0.5\SunspotPhase2Tlog\data_phase2\d_estimates_by_phase\M1_nonoverlap_counts_by_W_G_phase.csv


,W,G,phase,n_selected
0,60,1,max,25
1,60,1,min,24
2,60,6,max,25
3,60,6,min,24
4,60,12,max,25
5,60,12,min,24
6,132,1,max,19
7,132,1,min,19
8,132,6,max,19
9,132,6,min,18


[STEP=37][INFO][BLOC_5.4.2] Audit des effectifs non chevauchants M1 min/max par (W,G,phase) terminé. Écrit: M1_nonoverlap_counts_by_W_G_phase.csv.
[METRIC][M1_nonoverlap_counts_rows] = 18 (step=37)


### Bloc 5.4.A – Sanity‑check visuel M1 (min vs max) pour (W = 60, G = 12)

**Objectif.**  
Vérifier visuellement, sans conclusion prématurée, la séparation entre `min` et `max` pour M1 aux valeurs de `k` où un signal a été détecté.

**Entrées.**
- `data_phase2/d_estimates_by_phase/M1_S0_per_window_all_WG.csv` (M1 par fenêtre, toutes phases)
- `data_phase2/d_estimates_by_phase/M1_min_max_nonoverlap_samples.csv` (fenêtres non chevauchantes par phase)

**Méthode.**
- Filtrer `(W=60, G=12)` et `cycle_phase ∈ {min, max}`.
- Restreindre aux fenêtres non chevauchantes (fichier 5.4).
- Visualiser des boxplots pour `k ∈ {8, 12, 16, 20}` (sous‑ensemble dans la plage significative).
- Afficher un tableau récapitulatif (n_min, n_max, moyennes, médianes, Cliff’s δ par k).
- Aucun fichier n’est écrit par défaut (option `SAVE_FIG=False` modifiable).

**Sorties (affichage).**
- Tableau récapitulatif par k.
- Figure avec boxplots min vs max (échelle commune).